## Process

In [1]:
import numpy as np
import torch
from torch.autograd import Variable

def CalculateSimilarty(k1,k2,error_tolerance=0.1):
    
    diff = k1  - k2
    diff[diff<=error_tolerance]=0
    
    return (np.sum(diff==0) /(k1.shape[0]*k1.shape[1]))

In [2]:
# Make a random input tensor
tensor = Variable(torch.randn(1,128,32,32))
tensor = tensor.detach().numpy()
print('input tensor shape: ', tensor.shape)

# Make a inital zero tensor for a mean tensor
t0 = np.zeros((tensor.shape[0], tensor.shape[1], tensor.shape[2], tensor.shape[3])) # (1,128,32,32)

# Sum every value of each feature maps
for i in range(tensor.shape[1]):
    t0 += tensor[0][i]

# Make a mean tensor
mean_tensor = t0/tensor.shape[1]

# Set the threshold (0~1)
threshold = 0.7
temp_sim_list = []
where_sim_exceed_list = []

# Calculate similarty between a mean tensor and each feature map tensors, with setted error tolerance
for i in range(tensor.shape[1]):
    t0 = tensor[0][i]
    sim = CalculateSimilarty(t0, mean_tensor, error_tolerance=0.1)
    temp_sim_list.append(sim)
    
# If similarty exceed the threshold, put the index of threshold in the empty list
for i in range(tensor.shape[1]):
    normalized_sim = (temp_sim_list[i] - min(temp_sim_list))/(max(temp_sim_list) - min(temp_sim_list))
    if normalized_sim > threshold:
        where_sim_exceed_list.append(i)

print('\nnumber of feature map which has high similarty with a mean tensor : ', len(where_sim_exceed_list))

# Delete the feature map which has high similarty with a mean tensor
pruned_tensor = np.delete(tensor, [index for index in where_sim_exceed_list], axis=1)

# numpy array -> torch tensor
pruned_tensor = torch.from_numpy(pruned_tensor)

print('\npruned tensor shape: ', pruned_tensor.shape)

input tensor shape:  (1, 128, 32, 32)

number of feature map which has high similarty with a mean tensor :  19

pruned tensor shape:  torch.Size([1, 109, 32, 32])


In [3]:
# Print the normalized similarity between the feature map number of the tensor (to be pruned) and the mean tensor
for i in range(tensor.shape[1]):
    normalized_sim = (temp_sim_list[i] - min(temp_sim_list))/(max(temp_sim_list) - min(temp_sim_list))
    print('feature map : %03d, similarity: %0.4f' % (i, normalized_sim))

feature map : 000, similarity: 0.7229
feature map : 001, similarity: 0.3133
feature map : 002, similarity: 0.3976
feature map : 003, similarity: 0.3253
feature map : 004, similarity: 0.3373
feature map : 005, similarity: 0.4458
feature map : 006, similarity: 0.5181
feature map : 007, similarity: 0.4337
feature map : 008, similarity: 0.8434
feature map : 009, similarity: 0.4940
feature map : 010, similarity: 0.9036
feature map : 011, similarity: 0.7349
feature map : 012, similarity: 0.4940
feature map : 013, similarity: 0.7590
feature map : 014, similarity: 0.4096
feature map : 015, similarity: 0.5422
feature map : 016, similarity: 0.5663
feature map : 017, similarity: 0.5663
feature map : 018, similarity: 0.4458
feature map : 019, similarity: 0.5060
feature map : 020, similarity: 0.7711
feature map : 021, similarity: 0.3976
feature map : 022, similarity: 0.6265
feature map : 023, similarity: 0.9639
feature map : 024, similarity: 0.3614
feature map : 025, similarity: 0.5301
feature map 

## Making Pruning function based on the above process

In [4]:
def pruning(tensor, threshold, error_tolerance):
    
    tensor = tensor.detach().numpy()

    t = np.zeros((tensor.shape[0], tensor.shape[1], tensor.shape[2], tensor.shape[3])) # (1,128,32,32)

    for i in range(tensor.shape[1]):
        t += tensor[0][i]

    mean_tensor = t/tensor.shape[1]
    
    temp_sim_list = []
    where_sim_exceed_list = []

    for i in range(tensor.shape[1]):
        t0 = tensor[0][i]
        sim = CalculateSimilarty(t0, mean_tensor, error_tolerance)
        temp_sim_list.append(sim)
    
    for i in range(tensor.shape[1]):
        normalized_sim = (temp_sim_list[i] - min(temp_sim_list))/(max(temp_sim_list) - min(temp_sim_list))
        if normalized_sim > threshold:
            where_sim_exceed_list.append(i)

    pruned_tensor = np.delete(tensor, [index for index in where_sim_exceed_list], axis=1)
    pruned_tensor = torch.from_numpy(pruned_tensor)
    
    return pruned_tensor

In [5]:
test_tensor = Variable(torch.randn(1,128,32,32))
print('test tensor shape: ', test_tensor.shape)

pruned_tensor = pruning(test_tensor, threshold=0.7, error_tolerance=0.9)
print('pruned tensor shape: ', pruned_tensor.shape)

test tensor shape:  torch.Size([1, 128, 32, 32])
pruned tensor shape:  torch.Size([1, 114, 32, 32])


In [6]:
pruned_tensor

tensor([[[[ 6.8276e-01, -3.2362e-01,  1.5856e-01,  ..., -4.5694e-02,
           -2.4110e-01,  2.6615e-01],
          [ 2.3591e+00,  5.8619e-02, -2.1951e+00,  ...,  5.1960e-01,
            6.5440e-01, -3.0105e-01],
          [ 7.5492e-01,  1.3746e+00,  7.2352e-01,  ...,  5.9667e-01,
           -1.5843e+00,  9.7456e-01],
          ...,
          [-1.0756e+00, -4.4399e-01, -7.2137e-02,  ...,  7.0271e-01,
           -1.7004e-01, -2.4777e-01],
          [ 6.2031e-01,  1.1454e+00,  1.5474e+00,  ...,  8.2495e-01,
           -9.2652e-01, -1.2143e+00],
          [ 4.9259e-01, -4.5404e-01, -5.8627e-01,  ..., -3.5105e-01,
            1.3708e+00,  2.6773e-01]],

         [[ 1.2791e+00, -2.0823e-01,  2.7541e-01,  ..., -4.0371e-01,
            4.0912e-01, -2.1592e-01],
          [ 1.0990e-01, -2.3167e-01,  4.9143e-01,  ...,  6.7083e-01,
            9.1086e-02, -4.9011e-02],
          [ 1.1220e-01, -2.4509e-01, -2.3001e-01,  ...,  2.2945e+00,
            7.3707e-01,  7.0211e-01],
          ...,
     

## Applying method on the basic model

In [7]:
import torch
import torch.nn as nn


class Standard(nn.Module):
    
    def __init__(self):
        super(Standard, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 128, kernel_size = 3, stride = 1, padding = 1)
        self.conv2 = nn.Conv2d(in_channels = self.conv1.out_channels, out_channels = 256, kernel_size = 3, stride = 1)
        
    def pruning(self, tensor, threshold, error_tolerance):
    
        tensor = tensor.detach().numpy()

        t = np.zeros((tensor.shape[0], tensor.shape[1], tensor.shape[2], tensor.shape[3])) # (1,128,32,32)

        for i in range(tensor.shape[1]):
            t += tensor[0][i]

        mean_tensor = t/tensor.shape[1]

        temp_sim_list = []
        where_sim_exceed_list = []

        for i in range(tensor.shape[1]):
            t0 = tensor[0][i]
            sim = CalculateSimilarty(t0, mean_tensor, error_tolerance)
            temp_sim_list.append(sim)

        for i in range(tensor.shape[1]):
            normalized_sim = (temp_sim_list[i] - min(temp_sim_list))/(max(temp_sim_list) - min(temp_sim_list))
            if normalized_sim > threshold:
                where_sim_exceed_list.append(i)

        pruned_tensor = np.delete(tensor, [index for index in where_sim_exceed_list], axis=1)
        pruned_tensor = torch.from_numpy(pruned_tensor)

        return pruned_tensor
        
    def forward(self, x):
        
        x = self.pruning(self.conv1(x), threshold=0.7, error_tolerance=0.1)
        #x = self.conv2(x) -> How??
        
        return x

## Calculating the output feature map's parameters after pruning

In [8]:
def what_is_size_of_tensor(net):
    y = net(Variable(torch.randn(1,3,32,32)))
    print(y.size())

model = Standard()    
what_is_size_of_tensor(model) # original output size = (1, 128, 32, 32)

torch.Size([1, 104, 32, 32])
